In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


## Select Object Indices of Interest

| Object Indices |[]()|[]()|
|---|---|---|
| __1.__ 002_master_chef_can | __8.__ 009_gelatin_box      | __15.__ 035_power_drill       |
| __2.__ 003_cracker_box     | __9.__ 010_potted_meat_can  | __16.__ 036_wood_block        |
| __3.__ 004_sugar_box       | __10.__ 011_banana          | __17.__ 037_scissors          |
| __4.__ 005_tomato_soup_can | __11.__ 019_pitcher_base    | __18.__ 040_large_marker      |
| __5.__ 006_mustard_bottle  | __12.__ 021_bleach_cleanser | __19.__ 051_large_clamp       |
| __6.__ 007_tuna_fish_can   | __13.__ 024_bowl            | __20.__ 052_extra_large_clamp |
| __7.__ 008_pudding_box     | __14.__ 025_mug             | __21.__ 061_foam_brick        |

In [2]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None, None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q, t

## YCB Dataset

In [3]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes
from object_pose_utils.datasets.feature_dataset import  FeatureDataset
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

feature_root = '/scratch/bokorn/results/posecnn_feat_all/'
feature_key = 'fc6'
feature_size = 4096

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.TRANSFORM_MATRIX,
                ]

ycb_dataset = YCBDataset(dataset_root, mode=mode,
                     object_list = object_list,
                     output_data = output_format,
                     resample_on_error = False,
                     add_syn_background = False,
                     add_syn_noise = False,
                     #use_posecnn_data = True,
                     #preprocessors = [InplaneRotator()],
                     #postprocessors = [ImageNormalizer()],
                     image_size = [640, 480], num_points=1000)

dataset = FeatureDataset(dataset_root = dataset_root,
                         feature_root = feature_root,
                         feature_key = feature_key,
                         return_pred = True,
                         mode = mode,
                         resample_on_error = False,
                         object_list = list(range(1,22)))

model_clouds = {}
for object_id in object_list:
    cloud_filename = '{}/models/{}/points.xyz'.format(dataset_root, dataset.classes[object_id])
    model_clouds[object_id] = np.loadtxt(cloud_filename)

## Distribution Estimators

In [4]:
from se3_distributions.losses.loglik_loss import evaluateFeature
from se3_distributions.models.compare_networks import SigmoidCompareNet

if(False):
    grid_vertices = torch.load(os.path.join(feature_root, 'grid',
        '{}_vertices.pt'.format(dataset.classes[1])))

    grid_features = {}
    for object_id in object_list:
        grid_features[object_id] = torch.load(os.path.join(feature_root, 'grid',
            '{}_{}_features.pt'.format(feature_key, dataset.classes[object_id])))

    #comp_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_comp/**/weights/checkpoint_*.pth'
    #comp_model_checkpoint = sorted(glob.glob(comp_model_checkpoint_pattern,
    #                                        recursive=True))[-1]
    comp_model_checkpoint = '/scratch/bokorn/results/log_lik/posecnn_fc6_comp/lr_1e-6/2019-09-08_22-59-43/weights/best_quat.pth'
    print(glob.glob('/'.join(comp_model_checkpoint.split('/')[:-1]) + '/*', recursive=True))

    comp_estimator = SigmoidCompareNet(feature_size, len(object_list))
    comp_estimator.load_state_dict(torch.load(comp_model_checkpoint))
    comp_estimator.cuda();
    comp_estimator.eval();


    def hist_comp(res, obj):
        feat, pred_q, pred_t = res
        lik_est = evaluateFeature(comp_estimator, obj, feat, grid_features)
        lik_est = to_np(lik_est.flatten())
        lik_est /= lik_est.sum()
        return lik_est, pred_q, pred_t

['/scratch/bokorn/results/log_lik/posecnn_fc6_reg/lr_1e-6/2019-09-06_14-09-38/weights/checkpoint_8000.pth',
 '/scratch/bokorn/results/log_lik/posecnn_fc6_reg/lr_1e-6/2019-09-06_14-09-38/weights/checkpoint_9000.pth',
 '/scratch/bokorn/results/log_lik/posecnn_fc6_reg/lr_1e-6/2019-09-06_17-54-28/weights/checkpoint_76000.pth']

In [5]:
from se3_distributions.losses.loglik_loss import evaluateFeature
from se3_distributions.models.compare_networks import SigmoidNet

if(False):
    grid_size = 3885

    #reg_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_reg/**/weights/checkpoint_*.pth'
    #reg_model_checkpoint = sorted(glob.glob(reg_model_checkpoint_pattern,
    #                                        recursive=True))[-1]
    reg_model_checkpoint = '/scratch/bokorn/results/log_lik/posecnn_fc6_reg/lr_1e-6/2019-09-06_17-54-28/weights/best_quat.pth'
    print(glob.glob('/'.join(reg_model_checkpoint.split('/')[:-1]) + '/*', recursive=True))

    reg_estimator = SigmoidNet(feature_size, len(object_list)*grid_size)
    reg_estimator.load_state_dict(torch.load(reg_model_checkpoint))
    reg_estimator.cuda();
    reg_estimator.eval();

    def hist_reg(res, obj):
        feat, pred_q, pred_t = res
        feat = feat.cuda()
        lik_est = evaluateFeature(reg_estimator, obj, feat, None)
        lik_est = to_np(lik_est.flatten())
        lik_est /= lik_est.sum()
        return lik_est, pred_q, pred_t


In [6]:
from se3_distributions.losses.loglik_loss import evaluateFeature
from se3_distributions.models.compare_networks import SigmoidNet

if(False):
    grid_size = 3885
    reg_model_idv_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_reg/{}/lr_1e-5/**/weights/final_*.pth'

    reg_estimator_idv = {}
    for object_id in object_list:
        reg_model_idv_checkpoint = sorted(glob.glob(reg_model_idv_checkpoint_pattern.format(object_id), recursive=True))[-1]
        reg_estimator_idv[object_id] = SigmoidNet(feature_size, grid_size)
        reg_estimator_idv[object_id].load_state_dict(torch.load(reg_model_idv_checkpoint))
        reg_estimator_idv[object_id].cuda();
        reg_estimator_idv[object_id].eval();

    def hist_reg_indv(res, obj):
        feat, pred_q, pred_t = res
        feat = feat.cuda()
        lik_est = evaluateFeature(reg_estimator_idv[obj.item()], obj, feat, None)
        lik_est = to_np(lik_est.flatten())
        lik_est /= lik_est.sum()
        return lik_est, pred_q, pred_t


In [7]:
from object_pose_utils.utils.pose_processing import quatAngularDiffBatch

if(False):
    #confusion_root = '/home/bokorn/data/confusion_matrices/'
    confusion_root = '/home/bokorn/src/se3_distributions/se3_distributions/evaluation_notebooks/conf_matices/posecnn/'
    confusion_eps = 0.000001

    confusion_matrices = {}
    for object_id in object_list:
        #confusion_matrices[object_id] = scio.loadmat(os.path.join(confusion_root, 
        #    "{0}_confusion_matrix.mat".format(object_id)))['loaded'] + confusion_eps
        confusion_matrices[object_id] = np.load(os.path.join(confusion_root, 
            "{0}_confusion_matrix.npy".format(object_id))) + confusion_eps
        
    def hist_conf(res, obj):
        feat, pred_q, pred_t = res
        dists = quatAngularDiffBatch(to_np(pred_q), to_np(grid_vertices))
        bin_idx = np.argmin(dists)
        lik_est = confusion_matrices[obj.item()][bin_idx]
        lik_est /= lik_est.sum()
        return lik_est, pred_q, pred_t


In [8]:
from object_pose_utils.utils.interpolation import BinghamInterpolation

if(True):
    from object_pose_utils.utils.bingham import isobingham_likelihood

    #sigmas_data = np.load('/home/bokorn/src/se3_distributions/se3_distributions/notebooks/fc6_sigma.npz', allow_pickle=True)
    sigmas_data = np.load('pc_new_sigmas.npz', allow_pickle=True)
    sigmas = sigmas_data['sigma'].item()

    def bing_fixed(res, obj):
        feat, pred_q, pred_t = res
        bingham = BinghamInterpolation(pred_q.unsqueeze(0).cuda(), 
                                       torch.ones(1).cuda(), 
                                       sigma=torch.Tensor([sigmas[obj.item()]]).cuda())
        return bingham, pred_q, pred_t

In [9]:
sigmas

{1: 220.25169290835154,
 2: 185.80987360157158,
 3: 116.9262349880116,
 4: 116.9262349880116,
 5: 289.1353315219115,
 6: 95.6400200220408,
 7: 254.69351221513153,
 8: 228.38230353351315,
 9: 108.79562436284999,
 10: 39.91198574929002,
 11: 74.35380505607002,
 12: 116.9262349880116,
 13: 143.23744366962998,
 14: 61.19820071526081,
 15: 130.0818393288208,
 16: 5.470166442510036,
 17: 61.19820071526081,
 18: 5.470166442510036,
 19: 39.91198574929002,
 20: 39.91198574929002,
 21: 151.3680542947916}

In [10]:
if(False):
    grid_vertices = torch.load(os.path.join(feature_root, 'grid',
        '{}_vertices.pt'.format(dataset.classes[1])))

    grid_features = {}
    for object_id in object_list:
        grid_features[object_id] = torch.load(os.path.join(feature_root, 'grid',
            '{}_{}_features.pt'.format(feature_key, dataset.classes[object_id])))
        
    def hist_cosine(res, obj):
        feat, pred_q, pred_t = res
        feat = feat.cuda()
        lik_est = torch.mm(grid_features[obj.item()].cuda(), feat.transpose(0,1)).flatten()
        grid_norm = grid_features[obj.item()].cuda().norm(dim=1)
        grid_norm[grid_norm==0] = 1
        feat_norm = feat.norm()
        if(feat_norm == 0):
            feat_norm = 1
        lik_est /= grid_norm*feat_norm
        lik_est /= lik_est.sum()
        return lik_est, pred_q, pred_t


    def hist_uniform(res, obj):
        feat, pred_q, pred_t = res
        lik_est = torch.ones(grid_size)
        lik_est /= lik_est.sum()
        return lik_est, pred_q, pred_t



In [11]:
from se3_distributions.models.bingham_networks import IsoBingham
from se3_distributions.losses.bingham_loss import isoLikelihood

if(False):
    #feature_size = 1024
    feature_size = 4096
    #iso_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_iso/**/weights/checkpoint_*.pth'
    #iso_model_checkpoint = sorted(glob.glob(iso_model_checkpoint_pattern,
    #                                        recursive=True))[-1]
    iso_model_checkpoint = '/scratch/bokorn/results/log_lik/posecnn_fc6_iso/lr_1e-6/2019-09-07_16-10-58/weights/best_quat.pth'
    print(glob.glob('/'.join(iso_model_checkpoint.split('/')[:-1]) + '/*', recursive=True))

    iso_estimator = IsoBingham(feature_size, len(object_list))
    iso_estimator.load_state_dict(torch.load(iso_model_checkpoint))
    iso_estimator.eval()
    iso_estimator.cuda()

    def bing_iso(res, obj):
        feat, pred_q, pred_t = res
        
        feat = torch.Tensor(feat).unsqueeze(0).cuda()
        mean_est = torch.Tensor(pred_q).unsqueeze(0).cuda()
        df_obj = torch.LongTensor(obj-1).unsqueeze(0).cuda()
        sig_est = iso_estimator(feat.unsqueeze(0).cuda(), df_obj)
        lik_est = isoLikelihood(mean_q=mean_est[0], 
                                sig=sig_est[0,0])
        
        return lik_est, pred_q, pred_t

In [12]:
from se3_distributions.models.bingham_networks import DuelBingham
from se3_distributions.losses.bingham_loss import duelLikelihood

if(False):

    
    #feature_size = 1024
    feature_size = 4096
    #duel_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_duel/**/weights/checkpoint_*.pth'
    #duel_model_checkpoint = sorted(glob.glob(duel_model_checkpoint_pattern,
    #                                         recursive=True))[-1]
    duel_model_checkpoint = '/scratch/bokorn/results/log_lik/posecnn_fc6_duel/lr_1e-6/2019-09-08_16-15-52/weights/best_quat.pth'
    print(glob.glob('/'.join(duel_model_checkpoint.split('/')[:-1]) + '/*', recursive=True))

    duel_estimator = DuelBingham(feature_size, len(object_list))
    duel_estimator.load_state_dict(torch.load(duel_model_checkpoint))
    duel_estimator.eval()
    duel_estimator.cuda()
    
    def bing_duel(res, obj):
        feat, pred_q, pred_t = res

        feat = torch.Tensor(feat).unsqueeze(0).cuda()
        mean_est = torch.Tensor(pred_q).unsqueeze(0).cuda()
        df_obj = torch.LongTensor(obj-1).unsqueeze(0).cuda()
        
        duel_est, z_est = duel_estimator(feat, df_obj)

        lik_est = duelLikelihood(mean_q=mean_est[0], 
                                 duel_q = duel_est[0,0],
                                 z=z_est[0])
        
        return lik_est, pred_q, pred_t

In [9]:
duel_model_checkpoint_pattern = '/scratch/bokorn/results/log_lik/posecnn_fc6_duel/**/weights/checkpoint_*.pth'
sorted(glob.glob(duel_model_checkpoint_pattern, recursive=True))

['/scratch/bokorn/results/log_lik/posecnn_fc6_duel/lr_1e-6/2019-09-07_15-59-19/weights/checkpoint_1000.pth',
 '/scratch/bokorn/results/log_lik/posecnn_fc6_duel/lr_1e-6/2019-09-07_16-09-23/weights/checkpoint_29000.pth',
 '/scratch/bokorn/results/log_lik/posecnn_fc6_duel/lr_1e-6/2019-09-08_16-15-52/weights/checkpoint_16000.pth']

In [13]:
lik_funcs = {#'hist_reg_fc6':hist_reg,
             #'hist_reg_indv_fc6_all':hist_reg_idv_global,
             #'hist_comp_fc6':hist_comp,
             #'hist_conf':hist_conf,
             #'hist_cosine':hist_cosine,
             #'hist_uniform':hist_uniform,
             'bing_fixed':bing_fixed,
             #'bing_iso':bing_iso,
             #'bing_duel':bing_duel,
             }

from se3_distributions.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance, meanShift
from object_pose_utils.utils.pose_error import add, adi
from quat_math import quaternion_matrix
from object_pose_utils.utils.interpolation import TetraInterpolation, BinghamInterpolation
from se3_distributions.losses.bingham_loss import isoLikelihood, duelLikelihood


tetra_interp = TetraInterpolation(2)

import pathlib

from tqdm import tqdm_notebook as tqdm

likelihood = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}

sym_angular_error = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}
add_error = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}
add_sym_error = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}

sym_angular_error_mode = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}
add_error_mode = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}
add_sym_error_mode = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}

lik_distribution = {k:{obj:{} for obj in object_list} for k in lik_funcs.keys()}

bad_data = []
with torch.no_grad():
    for j, data in enumerate(tqdm(dataset)):
        obj, quat, trans, _ = ycb_dataset[j]
        _, feat, quat_gt, quat_pred = data
        posecnn_path = '{}/data/{}-posecnn.mat'.format(dataset.dataset_root, dataset.getPath(j))
        posecnn_data = scio.loadmat(posecnn_path)
        pred_q, pred_t = getPoseCNNQuat(posecnn_data, obj.item())
        
        if(len(obj) == 0):
            bad_data.append(j)
            continue

        if(pred_q is None):
            continue
        feat = feat.unsqueeze(0)
        res = (feat, torch.tensor(pred_q).float(), pred_t)

        sym_axis, sym_ang = getYCBSymmeties(obj.item())
        trans = to_np(trans)

        for k, func in lik_funcs.items():            
            lik_est, q_est, t_est = func(res, obj)

            if(type(lik_est) in [BinghamInterpolation, isoLikelihood, duelLikelihood]):
                lik = lik_est(quat.unsqueeze(0).cuda()).item()
                q_mode = q_est
            else:
                if(type(lik_est) is torch.Tensor):
                    lik_est = to_np(lik_est.flatten())
                tetra_interp.setValues(lik_est)
                lik = tetra_interp.smooth(to_np(quat)).item()    
                q_mode = grid_vertices[np.argmax(lik_est)]
                #v_shift = meanShift(q_mode.cuda(), grid_vertices.cuda(), dist_est.cuda(),
                #                    sigma=np.pi/9, max_iter = 100)

            mat = quaternion_matrix(quat)

            err_ang = symmetricAngularDistance(torch.Tensor(q_est).unsqueeze(0), 
                                               quat.unsqueeze(0),
                                               sym_axis, sym_ang).item()*180/np.pi
            mat_est = quaternion_matrix(q_est)
            err_add = add(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                          model_clouds[obj.item()])
            err_adi = adi(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                          model_clouds[obj.item()])

            err_ang_mode = symmetricAngularDistance(torch.Tensor(q_mode).unsqueeze(0), 
                                                    quat.unsqueeze(0),
                                                    sym_axis, sym_ang).item()*180/np.pi

            mat_mode = quaternion_matrix(q_mode)
            err_add_mode = add(mat[:3,:3], trans, mat_mode[:3,:3], t_est, 
                               model_clouds[obj.item()])
            err_adi_mode = adi(mat[:3,:3], trans, mat_mode[:3,:3], t_est, 
                               model_clouds[obj.item()])

            likelihood[k][obj.item()][j]=lik  
            sym_angular_error[k][obj.item()][j]=err_ang
            add_error[k][obj.item()][j]=err_add
            add_sym_error[k][obj.item()][j]=err_adi

            sym_angular_error_mode[k][obj.item()][j]=err_ang_mode
            add_error_mode[k][obj.item()][j]=err_add_mode
            add_sym_error_mode[k][obj.item()][j]=err_adi_mode

            lik_distribution[k][obj.item()][j]=lik_est
        
np.savez('ral_results/final_pcnn_single_img_{}_results.npz'.format('_'.join(list(lik_funcs.keys()))), 
         likelihood=likelihood,
         sym_angular_error=sym_angular_error,
         add_error=add_error,
         add_sym_error=add_sym_error,
         sym_angular_error_mode=sym_angular_error_mode,
         add_error_mode=add_error_mode,
         add_sym_error_mode=add_sym_error_mode,
        )
np.savez('ral_results/final_pcnn_single_img_{}_dists.npz'.format('_'.join(list(lik_funcs.keys()))), 
         lik_distribution=lik_distribution,
        )

Exception on index 3096: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0050/000505_005_tomato_soup_can_feat.npz'
Exception on index 3097: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0050/000509_005_tomato_soup_can_feat.npz'
Exception on index 3098: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0050/000558_005_tomato_soup_can_feat.npz'
Exception on index 3099: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0050/000588_005_tomato_soup_can_feat.npz'
Exception on index 3100: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0050/000605_005_tomato_soup_can_feat.npz'
Exception on index 3101: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0050/000610_005_tomato_soup_can_feat.npz'
Exception on index 3102: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn

Exception on index 6871: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0059/000206_010_potted_meat_can_feat.npz'
Exception on index 12400: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0048/000653_051_large_clamp_feat.npz'
Exception on index 12401: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0048/000656_051_large_clamp_feat.npz'
Exception on index 12404: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0048/000685_051_large_clamp_feat.npz'
Exception on index 12405: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0048/000694_051_large_clamp_feat.npz'
Exception on index 12406: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0048/000733_051_large_clamp_feat.npz'
Exception on index 12412: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data

Exception on index 13618: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0057/001101_052_extra_large_clamp_feat.npz'
Exception on index 13619: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0057/001108_052_extra_large_clamp_feat.npz'
Exception on index 13620: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0057/001116_052_extra_large_clamp_feat.npz'
Exception on index 13621: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0057/001118_052_extra_large_clamp_feat.npz'
Exception on index 13622: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0057/001131_052_extra_large_clamp_feat.npz'
Exception on index 13624: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0057/001135_052_extra_large_clamp_feat.npz'
Exception on index 13625: [Errno 2] No such file or directory: '/scratch/bok

In [228]:
lik_est

array([2.4575042e-05, 2.3760236e-04, 6.2941962e-07, ..., 1.6985217e-06,
       1.5288599e-07, 1.7682225e-06], dtype=float32)